# Pulling Data with ALPACA-py SDK

* Link to SDK documentation: https://alpaca.markets/docs/python-sdk/index.html
* Link to REST API documentation: https://alpaca.markets/docs/introduction/#rest-sse-and-websockets

In [1]:
! pip install alpaca-py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.6/95.6 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 22.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.2/75.2 kB 9.8 MB/s eta 0:00:00
  Attempting uninstall: msgpack
    Found existing installation: msgpack 1.0.2
    Uninstalling msgpack-1.0.2:
      Successfully uninstalled msgpack-1.0.2
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.2
    Uninstalling typing-extensions-3.10.0.2:
      Successfully uninstalled typing-extensions-3.10.0.2


In [1]:
ALPACA_URL = "https://data.alpaca.markets"


import json 

#get Keys
f = open("/Users/austinpaxton/Documents/credentials/ALPACA_cred.json")
data = json.load(f)

headers = {
    'content-type': 'application/json',
    "APCA-API-KEY-ID": data["KEY"],
    "APCA-API-SECRET-KEY": data["SECRET_KEY"]
}

headers_test = {
                'content-type': 'application/json',
                'Apca-Api-Key-Id': data["KEY"],
                'Apca-Api-Secret-Key': data["SECRET_KEY"]
               }



In [3]:
#request Historical data for spy
import requests

symbol = "SPY"

daily_url = f"{ALPACA_URL}/v2/stocks/{symbol}/bars?timeframe=1Day"

response = requests.get(daily_url, headers=headers)
print(response)
print(response.content)

<Response [200]>
b'{"bars":[{"t":"2023-03-22T04:00:00Z","o":398.73,"h":402.49,"l":395.67,"c":398.94,"v":75249260,"n":665068,"vw":399.035195}],"symbol":"SPY","next_page_token":null}'


# Get Latest Quotes for multiple symbols

In [4]:


from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data.requests import StockLatestQuoteRequest

# keys required for stock historical data client
client = StockHistoricalDataClient(data["KEY"], data["SECRET_KEY"])

# multi symbol request - single symbol is similar
multisymbol_request_params = StockLatestQuoteRequest(symbol_or_symbols=["SPY", "AAPL"])

latest_quotes = client.get_stock_latest_quote(multisymbol_request_params)

print(latest_quotes)

{'AAPL': {   'ask_exchange': 'V',
    'ask_price': 160.38,
    'ask_size': 1.0,
    'bid_exchange': 'V',
    'bid_price': 160.3,
    'bid_size': 1.0,
    'conditions': ['R'],
    'symbol': 'AAPL',
    'tape': 'C',
    'timestamp': datetime.datetime(2023, 3, 22, 19, 24, 53, 210754, tzinfo=datetime.timezone.utc)}, 'SPY': {   'ask_exchange': 'V',
    'ask_price': 403.0,
    'ask_size': 1.0,
    'bid_exchange': 'V',
    'bid_price': 390.25,
    'bid_size': 1.0,
    'conditions': ['R'],
    'symbol': 'SPY',
    'tape': 'B',
    'timestamp': datetime.datetime(2023, 3, 22, 19, 24, 53, 180184, tzinfo=datetime.timezone.utc)}}


# Get Historical Data for SPY

In [3]:
from alpaca.data.requests import StockBarsRequest
from alpaca.data.timeframe import TimeFrame
from alpaca.data.timeframe import TimeFrameUnit


SPY_request = StockBarsRequest(symbol_or_symbols=["SPY"], 
                               timeframe=TimeFrame.Day,
                                start="2008-01-01 00:00:00",
                              limit=10000)

SPY_bars = client.get_stock_bars(request_params = SPY_request) #use client initiated in above cell
print(SPY_bars.json())


    

NameError: name 'client' is not defined

In [ ]:
SPY_df = SPY_bars.df
display(SPY_df)
print(len(SPY_df))
print(type(SPY_df))

# News not implemented yet on SDK so use REST API

## Test News for 1 Symbol

In [35]:
import requests
import json
symbols = 'TSLA'
start_date = "2018-09-01T00:00:00Z"
end_date = "2019-09-30T11:59:59Z"
url = f'https://data.alpaca.markets/v1beta1/news?start={start_date}&end={end_date}&symbols={symbols}'
headers = {'content-type': 'application/json', 'Apca-Api-Key-Id': data["KEY"], 'Apca-Api-Secret-Key': data["SECRET_KEY"]}
response = requests.get(url, headers=headers)
response_dict = json.loads(response.text)

print(response_dict["news"][9])

{'author': 'Benzinga Newsdesk', 'content': '', 'created_at': '2019-09-26T14:30:31Z', 'headline': "'Tesla starts wider release of V10, and it finally includes Spotify' -Electrek", 'id': 14497493, 'images': [], 'source': '', 'summary': '', 'symbols': ['SPOT', 'TSLA'], 'updated_at': '2019-09-26T14:30:31Z', 'url': 'https://www.benzinga.com/node/14497493'}


In [36]:
print(response_dict)

{'news': [{'author': 'Charles Gross', 'content': '', 'created_at': '2019-09-28T22:59:07Z', 'headline': 'First Report Of Accident With Tesla Smart Summon Emerges', 'id': 14509938, 'images': [], 'source': '', 'summary': '', 'symbols': ['TSLA'], 'updated_at': '2019-09-28T22:59:07Z', 'url': 'https://www.benzinga.com/node/14509938'}, {'author': 'Nelson Hem', 'content': '', 'created_at': '2019-09-28T19:22:32Z', 'headline': "Barron's Picks And Pans: American Airlines, Chipotle, GM, Wells Fargo, More", 'id': 14509839, 'images': [{'size': 'large', 'url': 'https://cdn.benzinga.com/files/imagecache/2048x1536xUP/images/story/2012/business-2651346_1920_5_1.jpg'}, {'size': 'small', 'url': 'https://cdn.benzinga.com/files/imagecache/1024x768xUP/images/story/2012/business-2651346_1920_5_1.jpg'}, {'size': 'thumb', 'url': 'https://cdn.benzinga.com/files/imagecache/250x187xUP/images/story/2012/business-2651346_1920_5_1.jpg'}], 'source': '', 'summary': '', 'symbols': ['AAL', 'AMGN', 'CMG', 'DUK', 'GM', 'NF

In [37]:
print(len(response_dict["news"]))  #Note: News API only pulls 10 items at a time
# where the first item is the enddate and the remaining 

10


## Test News for Multiple Symbols

In [46]:
symbols = "TSLA,GOOG,MSFT"
start_date = "2018-09-01T00:00:00Z"
end_date = "2022-09-30T11:59:59Z"
url = f'https://data.alpaca.markets/v1beta1/news?start={start_date}&end={end_date}&symbols={symbols}'
headers = {'content-type': 'application/json', 'Apca-Api-Key-Id': data["KEY"], 'Apca-Api-Secret-Key': data["SECRET_KEY"]}
response = requests.get(url, headers=headers)
response_dict = json.loads(response.text)

print(f'Number of Articles: {len(response_dict["news"])}')
print(response_dict)

Number of Articles: 10
{'news': [{'author': 'Shivdeep Dhaliwal', 'content': '', 'created_at': '2022-09-30T10:34:36Z', 'headline': "Elon Musk Texts Reveal Frustration: 'Fixing Twitter By Chatting With Parag Won't Work'", 'id': 29089688, 'images': [{'size': 'large', 'url': 'https://cdn.benzinga.com/files/imagecache/2048x1536xUP/images/story/2022/09/30/shutterstock_2149752817.jpg'}, {'size': 'small', 'url': 'https://cdn.benzinga.com/files/imagecache/1024x768xUP/images/story/2022/09/30/shutterstock_2149752817.jpg'}, {'size': 'thumb', 'url': 'https://cdn.benzinga.com/files/imagecache/250x187xUP/images/story/2022/09/30/shutterstock_2149752817.jpg'}], 'source': 'benzinga', 'summary': 'A dump of text messages featuring Elon Musk’s interactions with Twitter Inc.\xa0(NYSE: TWTR) founder Jack Dorsey and its CEO Parag Agrawal and a host of other big', 'symbols': ['TSLA', 'TWTR'], 'updated_at': '2022-09-30T10:34:36Z', 'url': 'https://www.benzinga.com/news/22/09/29089688/elon-musk-texts-reveal-frust

Since the API returns 10 articles, for multi-symbol and single requests, we will loop through a list of tickers and use single requests so we can add the stock we are requesting as a column to the dictionary.

In [48]:
#test response when no articles are found
symbols = "TSLA"
start_date = "1990-09-01T00:00:00Z"
end_date = "1991-09-30T11:59:59Z"
url = f'https://data.alpaca.markets/v1beta1/news?start={start_date}&end={end_date}&symbols={symbols}'
headers = {'content-type': 'application/json', 'Apca-Api-Key-Id': data["KEY"], 'Apca-Api-Secret-Key': data["SECRET_KEY"]}
response = requests.get(url, headers=headers)
response_dict = json.loads(response.text)
print(response_dict)

{'news': [], 'next_page_token': None}


In [25]:
news_dict = {
    "author": [],
    "content": [],
    "created_at": [],
    "headline": [],
    "id": [],
    "source": [],
    "summary": [],
    "symbols": [],
    "updated_at": [],
    "url": [],
    "requested_TICKER": []
}

for article in response_dict["news"]:
    news_dict["author"].append(article["author"])
    news_dict["content"].append(article["content"])
    news_dict["created_at"].append(article["created_at"])
    news_dict["headline"].append(article["headline"])
    news_dict["id"].append(article["id"])
    news_dict["source"].append(article["source"])
    news_dict["summary"].append(article["summary"])
    news_dict["symbols"].append(article["symbols"])
    news_dict["updated_at"].append(article["updated_at"])
    news_dict["url"].append(article["url"])
    
    

pd.DataFrame(news_dict)

,author,content,created_at,headline,id,source,summary,symbols,updated_at,url
0,Charles Gross,,2019-09-28T22:59:07Z,First Report Of Accident With Tesla Smart Summ...,14509938,,,[TSLA],2019-09-28T22:59:07Z,https://www.benzinga.com/node/14509938
1,Nelson Hem,,2019-09-28T19:22:32Z,"Barron's Picks And Pans: American Airlines, Ch...",14509839,,,"[AAL, AMGN, CMG, DUK, GM, NFLX, PTON, TSLA, WF...",2019-09-28T19:22:33Z,https://www.benzinga.com/node/14509839
2,Benzinga Newsdesk,,2019-09-27T19:10:28Z,'Tesla To Set Model 3 Sales Record In The Neth...,14507605,,,[TSLA],2019-09-27T19:10:29Z,https://www.benzinga.com/node/14507605
3,Sebastian Brown,,2019-09-27T19:08:29Z,A Look At Benzinga Pro's Most-Searched Tickers...,14506031,,,"[BYND, CCL, DIS, MRNS, MU, PRPO, PTON, RAD, SQ...",2019-09-27T19:08:29Z,https://www.benzinga.com/node/14506031
4,Shanthi Rexaline,,2019-09-27T17:51:36Z,Tesla Analyst: September Rally In Deliveries C...,14499089,,,"[NIO, TSLA]",2019-09-27T17:51:37Z,https://www.benzinga.com/node/14499089
5,Benzinga Newsdesk,,2019-09-27T13:57:36Z,Benzinga Pro's Top 10 Most-Searched Tickers Fo...,14504871,,,"[BYND, CCL, DIS, MRNS, MU, PRPO, PTON, RAD, SQ...",2019-09-27T13:57:37Z,https://www.benzinga.com/node/14504871
6,Benzinga Newsdesk,,2019-09-26T19:19:41Z,Tesla To LAunch Tesla Theater With IQIYI And T...,14500309,,,"[IQ, SPOT, TSLA]",2019-09-26T19:19:41Z,https://www.benzinga.com/node/14500309
7,Benzinga Newsdesk,,2019-09-26T17:19:27Z,"'Tesla 'has a shot' at record 100,000 cars thi...",14499420,,,[TSLA],2019-09-26T17:19:27Z,https://www.benzinga.com/node/14499420
8,Benzinga Newsdesk,,2019-09-26T16:14:52Z,'Tesla produces complete Model 3 body-in-white...,14498733,,,[TSLA],2019-09-26T16:14:53Z,https://www.benzinga.com/node/14498733
9,Benzinga Newsdesk,,2019-09-26T14:30:31Z,"'Tesla starts wider release of V10, and it fin...",14497493,,,"[SPOT, TSLA]",2019-09-26T14:30:31Z,https://www.benzinga.com/node/14497493


In [89]:
#create function to get recursively pull news articles bc we are limited to 10 per request
import datetime

def pull_news(ticker, start_date, end_date, headers, news_dict, rate_limit=10):
        print(news_dict)
        url = f'https://data.alpaca.markets/v1beta1/news?start={start_date}&end={end_date}&symbols={symbol}'
        response = requests.get(url, headers=headers)
#         print(f'________________________________Response: {response}_____________________________________')
        rate_limit -= 1
#         print(f"________________________________Rate Limit: {rate_limit}________________________________")
        response_dict = json.loads(response.text)
        if response_dict["news"]:
            for article in response_dict["news"]:
                news_dict["author"].append(article["author"])
                news_dict["content"].append(article["content"])
                news_dict["created_at"].append(article["created_at"])
                news_dict["headline"].append(article["headline"])
                news_dict["id"].append(article["id"])
                news_dict["source"].append(article["source"])
                news_dict["summary"].append(article["summary"])
                news_dict["symbols"].append(article["symbols"])
                news_dict["updated_at"].append(article["updated_at"])
                news_dict["url"].append(article["url"])
                news_dict["requested_TICKER"].append(ticker)
                
            end_date=response_dict["news"][-1]["created_at"]
            
            if rate_limit>0:
                print(f"\n\n{ticker} || {start_date} || {end_date} || {rate_limit}\n\n")
                pull_news(ticker, start_date, end_date, headers, news_dict, rate_limit-1)   
            else:
                return ticker, start_date, end_date, headers, news_dict, rate_limit

In [92]:

start_date = "2018-09-01T00:00:00Z"
end_date = "2022-09-30T11:59:59Z"
news_dict = {
    "author": [],
    "content": [],
    "created_at": [],
    "headline": [],
    "id": [],
    "source": [],
    "summary": [],
    "symbols": [],
    "updated_at": [],
    "url": [],
    "requested_TICKER": []
}

print(pull_news("TSLA", start_date, end_date, headers, news_dict, 10))



{'author': [], 'content': [], 'created_at': [], 'headline': [], 'id': [], 'source': [], 'summary': [], 'symbols': [], 'updated_at': [], 'url': [], 'requested_TICKER': []}


ConnectionError: HTTPSConnectionPool(host='data.alpaca.markets', port=443): Max retries exceeded with url: /v1beta1/news?start=2018-09-01T00:00:00Z&end=2022-09-30T11:59:59Z&symbols=SPY (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fd42a12f2e0>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))

In [9]:
import pandas as pd

SPY_names = pd.read_csv("/Users/austinpaxton/Documents/ML Bootcamp/Capstone/SPY_companies.csv")

NameError: name 'test_dict' is not defined